In [ ]:
#%load_ext jupyter_black

In [ ]:
import datetime as dt
import math
import os
import random
import sys
import time

import keras
import numpy as np
import pandas as pd
import sklearn
import talib
import tensorflow as tf
import tensorflow_addons as tfa
import yfinance as yf
from fastai.tabular.core import df_shrink
from matplotlib import pyplot as plt
#from tensorflow.keras.metrics import *
from trading_modules.create_backtest import (
    VaR,
    average_drawdown,
    average_drawdown_squared,
    beta,
    burke_ratio,
    calmar_ratio,
    classification_report_display,
    conditional_sharpe_ratio,
    confusion_matrix_display,
    cVaR,
    excess_var,
    financial_evaluation,
    gain_loss_ratio,
    hpm,
    information_ratio,
    kappa_three_ratio,
    lpm,
    max_drawdown,
    modigliani_ratio,
    omega_ratio,
    plain_classification_report,
    plot_charts,
    plot_init,
    plot_tables,
    second_2_minute_converter,
    sharpe_ratio,
    sortino_ratio,
    sterling_ratio,
    treynor_ratio,
    upside_potential_ratio,
    volatility,
)
from trading_modules.create_data import (
    CMF,
    HMA,
    adjustPrices,
    create_2d_data,
    create_all_indicators_in_talib,
    create_labels,
    number_null_and_nan,
    one_hot,
    pltcolor,
    reverse_one_hot,
    show_label_distribution,
    show_price_and_labels,
    show_prices,
    trendNormalizePrices,
)
from trading_modules.create_neural_networks import (
    create_model_CNN_2D,
    create_model_GRU,
    create_model_LSTM,
    create_model_MLP,
    model_ho,
    model_train_test,
    show_epoch_and_score,
)
from trading_modules.create_strategy import (
    create_buy_and_hold_predictions,
    create_bollinger_bands_predictions,
    create_ema_crossover_predictions,
    create_random_predictions,
    create_rsi_predictions,
)
from trading_modules.feaure_engineering import feature_selection, scaling
from trading_modules.configurations import shift_predictions

<h1 style="font-size:40px;"> <center> CONSTANTS </center> </h1>

In [ ]:
SEED = 42
TRAIN_TEST_EPOCH = 30
HO_EPOCH = 10
HO_TRIALS = 10
FINAL_MODEL_EPOCH = 50
MAX_ROW_AND_COLUMN_2_DISPLAY = 100
LENGTH = 30
METRIC="precision" # precision, recall, f1_score
TICKER = "SPY"
BENCHMARK_TICKER = "SPY"
START_DATE = "2009-05-20"
END_DATE = "2023-01-01"
INTERVAL = "1d"
RUN_MLP = FALSE
RUN_LSTM = FALSE
RUN_GRU = FALSE
RUN_CNN = FALSE

In [ ]:
pd.set_option("display.max_rows", MAX_ROW_AND_COLUMN_2_DISPLAY)
pd.set_option("display.max_columns", MAX_ROW_AND_COLUMN_2_DISPLAY)

<h1 style="font-size:40px;"> <center> DATA PREPROCESSING </center> </h1>

In [ ]:
prices = yf.download(
    TICKER,
    start=START_DATE,
    end=END_DATE,
    interval=INTERVAL,
    progress=False,
    auto_adjust=True,
)
prices

In [ ]:
print(f"{prices.memory_usage().sum()/(1024**2)} MB is used before df_shrink")
prices = df_shrink(prices, int2uint=True)
print(f"{prices.memory_usage().sum()/(1024**2)} MB is used after df_shrink")

In [ ]:
show_prices(ticker=TICKER, df=prices.iloc[150:, :], desc=f"Close Price of {TICKER}")

<h1 style="font-size:30px;"> <center> Trend Normalize and Visualize </center> </h1>

In [ ]:
tn_prices = trendNormalizePrices(prices)
tn_prices

In [ ]:
show_prices(
    ticker=TICKER, df=tn_prices.iloc[150:, :], desc=f"Close TN-Price of {TICKER}"
)

<h1 style="font-size:30px;"> <center> Adding Technical Indicators </center> </h1>

In [ ]:
prices_and_indicators = create_all_indicators_in_talib(
    df=tn_prices, periods=list(range(7, 30))
)

In [ ]:
prices_and_indicators.head(5)

<h1 style="font-size:30px;"> <center> Data Labeling </center> </h1>

In [ ]:
prices_and_indicators_with_label = create_labels(prices_and_indicators)
prices_and_indicators_with_label.head(5)

In [ ]:
show_label_distribution(prices_and_indicators_with_label)

In [ ]:
prices_with_label = create_labels(prices)
show_price_and_labels(
    ticker=TICKER, df=prices_with_label, desc=f"Close Price with Labels of {TICKER}"
)

In [ ]:
tn_prices_with_label = create_labels(tn_prices)
show_price_and_labels(
    ticker=TICKER,
    df=tn_prices_with_label,
    desc=f"Close TN-Price with Labels of {TICKER}",
)

<h1 style="font-size:30px;"> <center> Creating Train & Test Data </center> </h1>

In [ ]:
prices_and_indicators_with_label.info()

In [ ]:
datas_1d = []

for i in range(5, 13):
    train = prices_and_indicators_with_label.loc[
        (prices_and_indicators_with_label.index >= f"{2009+i-5}-12")
        & (prices_and_indicators_with_label.index <= f"{2010+i}")
    ]
    test = prices_and_indicators_with_label.loc[
        (prices_and_indicators_with_label.index >= f"{2009+i}-12")
        & (prices_and_indicators_with_label.index <= f"{2010+i+1}")
    ]
    datas_1d.append([train, test])

In [ ]:
datas_1d[0][0].head(5)

<h1 style="font-size:30px;"> <center> Feature Selection </center> </h1>

In [ ]:
datas_1d[0][0].head(5)

In [ ]:
datas_1d, selected_features = feature_selection(datas_1d)

In [ ]:
print(selected_features)

In [ ]:
datas_1d[0][0].head(5)

<h1 style="font-size:30px;"> <center> Feature Scaling </center> </h1>

In [ ]:
datas_1d = scaling(datas_1d)

In [ ]:
datas_1d[0][0].head(5)

In [ ]:
datas_1d[0][1].head(5)

<h1 style="font-size:30px;"> <center> Controling Null Values </center> </h1>

In [ ]:
total_na_count = 0
for data in datas_1d:
    total_na_count += number_null_and_nan(data[0])
    total_na_count += number_null_and_nan(data[1])
print(f"Total null and nan values = {total_na_count}")

In [ ]:
test_indices = []
for data in datas_1d:
    test_indices.extend(data[1][29:].index)

<h1 style="font-size:40px;"> <center> OTHER STRATEGIES </center> </h1>

<h1 style="font-size:30px;"> Buy & Hold </h1>

In [ ]:
buy_and_hold_prediction = create_buy_and_hold_predictions(len(test_indices))
buy_and_hold_prediction

<h1 style="font-size:30px;"> Random Trades </h1>

In [ ]:
random_prediction = create_random_predictions(prices.loc[test_indices])
random_prediction

<h1 style="font-size:30px;"> RSI </h1>

In [ ]:
rsi_prediction = create_rsi_predictions(
    prices_and_indicators.loc[test_indices], period=14, buy_value=40, sell_value=60
)
rsi_prediction

<h1 style="font-size:30px;"> EMA Cross-over </h1>

In [ ]:
ema_crossover_prediction = create_ema_crossover_predictions(
    prices_and_indicators.loc[test_indices]
)
ema_crossover_prediction

<h1 style="font-size:30px;"> Bollinger Bands </h1>

In [ ]:
bollinger_bands_prediction = create_bollinger_bands_predictions(
    prices.loc[test_indices]
)
bollinger_bands_prediction

<h1 style="font-size:40px;"> <center> INITIAL MODELS </center> </h1>

<h1 style="font-size:30px;"> Matrix Data Creation </h1>

In [ ]:
datas_2d = create_2d_data(datas_1d, LENGTH)

<h1 style="font-size:30px;"> <center> MLP </center> </h1>

In [ ]:
if RUN_MLP:
    results = model_train_test("MLP", datas_1d, epochs=TRAIN_TEST_EPOCH, metric=METRIC)
    print(f"\nMean {METRIC} score: {results[1]}\n")
    print(f"Completed in {results[2]} minutes")
    history = results[3]
    show_epoch_and_score(
        history,
        [
            "loss",
            f"{METRIC}",
        ],
    )

<h1 style="font-size:30px;"> <center> LSTM </center> </h1>

In [ ]:
if RUN_LSTM:  
    results = model_train_test("LSTM", datas_2d, epochs=TRAIN_TEST_EPOCH)
    print(f"\nMean f1-macro score: {results[1]}\n")
    print(f"Completed in {results[2]} minutes")
    history = results[3]
    show_epoch_and_score(
        history,
        [
            "loss",
            "f1_score",
        ],
    )

<h1 style="font-size:30px;"> <center> GRU </center> </h1>

In [ ]:
if RUN_GRU:     
    results = model_train_test("GRU", datas_2d, epochs=TRAIN_TEST_EPOCH)
    print(f"\nMean f1-macro score: {results[1]}\n")
    print(f"Completed in {results[2]} minutes")
    history = results[3]
    show_epoch_and_score(
        history,
        [
            "loss",
            "f1_score",
        ],
    )

<h1 style="font-size:30px;"> <center> CNN </center> </h1>

In [ ]:
if RUN_CNN:  
    results = model_train_test("CNN_2D", datas_2d, epochs=TRAIN_TEST_EPOCH)
    print(f"\nMean f1-macro score: {results[1]}\n")
    print(f"Completed in {results[2]} minutes")
    history = results[3]
    show_epoch_and_score(
        history,
        [
            "loss",
            "f1_score",
        ],
    )

<h1 style="font-size:40px;"> <center> HYPERPARAMETER TUNING </center> </h1>

<h1 style="font-size:30px;"> <center> MLP </center> </h1>

In [ ]:
if RUN_MLP:
    mlp_parameter_space = {
        "activation_func": ["relu", "selu", "swish"],
        "dropout_rate": [0.1, 0.2, 0.3],
        "optimizer_algo": ["adam", "adadelta", "rmsprop"],
        "batch_size": [32, 64, 256],
    }
    best_mlp_parameters = model_ho(
        "MLP",
        datas_1d,
        epochs=HO_EPOCH,
        parameter_space=mlp_parameter_space,
        metric=METRIC,
        trial_number=50, #HO_TRIALS
    )

<h1 style="font-size:30px;"> <center> LSTM </center> </h1>

In [ ]:
if RUN_LSTM:
    lstm_parameter_space = {
        "activation_func": ["tanh", "swish"],
        "dropout_rate": [0.2, 0.3, 0.4],
        "optimizer_algo": ["adam", "adadelta", "rmsprop"],
        "batch_size": [32, 64, 256],
    }
    best_lstm_parameters = model_ho(
        "LSTM",
        datas_2d,
        epochs=HO_EPOCH,
        parameter_space=lstm_parameter_space,
        trial_number=HO_TRIALS,
    )

<h1 style="font-size:30px;"> <center> GRU </center> </h1>

In [ ]:
if RUN_GRU:
    gru_parameter_space = {
        "activation_func": ["tanh", "swish"],
        "dropout_rate": [0.2, 0.3, 0.4],
        "optimizer_algo": ["adam", "adadelta", "rmsprop"],
        "batch_size": [32, 64, 256],
    }
    best_gru_parameters = model_ho(
        "GRU",
        datas_2d,
        epochs=HO_EPOCH,
        parameter_space=gru_parameter_space,
        trial_number=HO_TRIALS,
    )

<h1 style="font-size:30px;"> <center> CNN </center> </h1>

In [ ]:
if RUN_CNN:
    cnn2d_parameter_space = {
        "activation_func": ["relu", "selu", "swish"],
        "dropout_rate": [0.3, 0.4, 0.5],
        "optimizer_algo": ["adam", "adadelta", "rmsprop"],
        "batch_size": [32, 64, 256],
    }
    best_cnn2d_parameters = model_ho(
        "CNN_2D",
        datas_2d,
        epochs=HO_EPOCH,
        parameter_space=cnn2d_parameter_space,
        trial_number=HO_TRIALS,
    )

<h1 style="font-size:40px;"> <center> FINAL MODEL CREATION </center> </h1>

<h1 style="font-size:30px;"> <center> MLP </center> </h1>

In [ ]:
if RUN_MLP:
    results = model_train_test(
        "MLP",
        datas_1d,
        epochs=FINAL_MODEL_EPOCH, parameters=best_mlp_parameters, metric=METRIC
    )
    history = results[3]
    show_epoch_and_score(
        history,
        [
            "loss",
            f"{METRIC}",
        ],
    )
    EPOCH_BEFORE_OVERFIT = 20
    mlp_predictions = model_train_test(
        "MLP",
        datas_1d,
        epochs=EPOCH_BEFORE_OVERFIT, parameters=best_mlp_parameters, metric=METRIC
    )[0]
    mlp_prediction = np.concatenate(mlp_predictions)
    mlp_prediction = shift_predictions(mlp_prediction)
    mlp_prediction

<h1 style="font-size:30px;"> <center> LSTM </center> </h1>

In [ ]:
if RUN_LSTM:
    results = model_train_test(
        "LSTM", datas_2d, epochs=FINAL_MODEL_EPOCH, parameters=best_lstm_parameters
    )
    history = results[3]
    show_epoch_and_score(
        history,
        [
            "loss",
            "f1_score",
        ],
    )
    EPOCH_BEFORE_OVERFIT = 20
    lstm_predictions = model_train_test(
        "LSTM", datas_2d, epochs=EPOCH_BEFORE_OVERFIT, parameters=best_lstm_parameters
    )[0]
    lstm_prediction = np.concatenate(lstm_predictions)
    lstm_prediction = shift_predictions(lstm_prediction)
    lstm_prediction

<h1 style="font-size:30px;"> <center> GRU </center> </h1>

In [ ]:
if RUN_GRU:
    results = model_train_test(
        "GRU", datas_2d, epochs=FINAL_MODEL_EPOCH, parameters=best_gru_parameters
    )
    history = results[3]
    show_epoch_and_score(
        history,
        [
            "loss",
            "f1_score",
        ],
    )
    EPOCH_BEFORE_OVERFIT = 20
    gru_predictions = model_train_test(
        "GRU", datas_2d, epochs=EPOCH_BEFORE_OVERFIT, parameters=best_gru_parameters
    )[0]
    gru_prediction = np.concatenate(gru_predictions)
    gru_prediction = shift_predictions(gru_prediction)
    gru_prediction

<h1 style="font-size:30px;"> <center> CNN-2D </center> </h1>

In [ ]:
if RUN_CNN:
    results = model_train_test(
        "CNN_2D",
        datas_2d,
        epochs=FINAL_MODEL_EPOCH,
        parameters=best_cnn2d_parameters,
    )
    history = results[3]
    show_epoch_and_score(
        history,
        [
            "loss",
            "f1_score",
        ],
    )
    EPOCH_BEFORE_OVERFIT = 20
    cnn2d_predictions = model_train_test(
        "CNN_2D", datas_2d, epochs=EPOCH_BEFORE_OVERFIT, parameters=best_cnn2d_parameters
    )[0]
    cnn2d_prediction = np.concatenate(cnn2d_predictions)
    cnn2d_prediction = shift_predictions(cnn2d_prediction)
    cnn2d_prediction

<h1 style="font-size:40px;"> <center> CLASSIFICATION EVALUATION </center> </h1>

In [ ]:
test_labels = prices_and_indicators_with_label.loc[test_indices]
test_labels.head(5)

<h1 style="font-size:30px;"> <center> MLP </center> </h1>

In [ ]:
if RUN_MLP:
    plain_classification_report(test_labels, mlp_prediction)
    classification_report_display(test_labels, mlp_prediction)
    confusion_matrix_display(test_labels, mlp_prediction)

<h1 style="font-size:30px;"> <center> LSTM </center> </h1>

In [ ]:
if RUN_LSTM:
    plain_classification_report(test_labels, lstm_prediction)
    classification_report_display(test_labels, lstm_prediction)
    confusion_matrix_display(test_labels, lstm_prediction)

<h1 style="font-size:30px;"> <center> GRU </center> </h1>

In [ ]:
if RUN_GRU:
    plain_classification_report(test_labels, gru_prediction)
    classification_report_display(test_labels, gru_prediction)
    confusion_matrix_display(test_labels, gru_prediction)

<h1 style="font-size:30px;"> <center> CNN-2D </center> </h1>

In [ ]:
if RUN_CNN:    
    plain_classification_report(test_labels, cnn2d_prediction)
    classification_report_display(test_labels, cnn2d_prediction)
    confusion_matrix_display(test_labels, cnn2d_prediction)

<h1 style="font-size:40px;"> <center> FINANCIAL EVALUATION </center> </h1>

In [ ]:
test_prices = prices.loc[test_indices]
test_prices

In [ ]:
configurations = {
    "ticker": TICKER,
    "benchmark_ticker": BENCHMARK_TICKER,
    "ohlcv": test_prices,
    "risk_free_rate": 0.05 / 252,
    "initial_capital": 1000,
    "commission": 1,
    "alpha": 0.05,
    "threshold": 0,
    "order": 1,
    "order_type": "market",  # "market" or "limit"
    "short": False,
    "short_fee": 1,
    "standard_take_profit": False,
    "trailing_take_profit": False,
    "take_profit_ratio": 100,
    "standard_stop_loss": False,
    "trailing_stop_loss": False,
    "stop_loss_ratio": 100,
    "leverage": 1,
    "miss_rate": 10,
    "show_initial_configuration": True,
    "show_tables": True,
    "show_charts": True,
    "show_time": True,
    "precision_point": 3,
}

<h1 style="font-size:30px;"> <center> Buy & Hold </center> </h1>

In [ ]:
configurations["predictions"] = buy_and_hold_prediction

metrics = financial_evaluation(**configurations)

<h1 style="font-size:30px;"> <center> Random Prediction </center> </h1>

In [ ]:
configurations["predictions"] = random_prediction

metrics = financial_evaluation(**configurations)

<h1 style="font-size:30px;"> <center> RSI </center> </h1>

In [ ]:
configurations["predictions"] = rsi_prediction

metrics = financial_evaluation(**configurations)

<h1 style="font-size:30px;"> <center> EMA Crossover </center> </h1>

In [ ]:
configurations["predictions"] = ema_crossover_prediction

metrics = financial_evaluation(**configurations)

<h1 style="font-size:30px;"> <center> Bollinger Bands </center> </h1>

In [ ]:
configurations["predictions"] = bollinger_bands_prediction

metrics = financial_evaluation(**configurations)

<h1 style="font-size:30px;"> <center> MLP </center> </h1>

In [ ]:
if RUN_MLP:
    configurations["predictions"] = mlp_prediction
    metrics = financial_evaluation(**configurations)

<h1 style="font-size:30px;"> <center> LSTM </center> </h1>

In [ ]:
if RUN_LSTM: 
    configurations["predictions"] = lstm_prediction
    metrics = financial_evaluation(**configurations)

<h1 style="font-size:30px;"> <center> GRU </center> </h1>

In [ ]:
if RUN_GRU:
    configurations["predictions"] = gru_prediction
    metrics = financial_evaluation(**configurations)

<h1 style="font-size:30px;"> <center> CNN </center> </h1>

In [ ]:
if RUN_CNN:
    configurations["predictions"] = cnn2d_prediction
    metrics = financial_evaluation(**configurations)